# So You're Going Outside: A Coronavirus Infection Risk Estimator For Leaving The House

## A list of all parameters, total 9:

**global:**
* p_i = probability that any given person is infected.
* p_{hu_vs} = probability that your hands are uninfected given that you touched a contaminated (with virus) surface
* p_air = the "fuck you" term for direct inhalation. probability of 
    breathing it from 1 breath of an infected person. let's call it 10^-6.

**situation-specific:**

*surfaces:*
* N_s = number of surfaces you touch while outside
* N_p_s = average number of people that touched the surfaces you're touching. 

*breathing*
* N_p_b = number of people that breathed on you
* N_b = the number of breath events. could take it as different per person, but don't have to.
* P_script = "projectiness" = 1 for a cough. estimated 0.3 for talking and 0.1 for wearing a mask. 
* r = the distance, in feet, that you're standing away from the breathing person

In [3]:
import numpy
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()
import scipy.stats
from __future__ import division

In [66]:
# define global parameters
# p_i
sf_population = 881549 # 2019 est: https://www.census.gov/quickfacts/sanfranciscocountycalifornia
confirmed_corona_cases = 1137 # April 18, 2020. https://missionlocal.org/2020/04/18-daily-figures-on-sf-coronavirus-cases-confirmed-cases/
confirmed_to_latent_multiplier = 10 # approximate, based on estimation of under-testing
c2l_to_asymptomatic_multiplier = 0.18 # source: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7078829/ 
p_i = (confirmed_corona_cases 
       * confirmed_to_latent_multiplier
       * c2l_to_asymptomatic_multiplier) / sf_population  

p_hu_vs = 0.5 # a highly approximate Fermi estimate.
p_air = 0.000001 # 10^-6, a highly approximate Fermi estimate.


In [124]:
def SYGOestimator(N_s, N_p_s, N_p_b, N_b, r, projectiness=0.3, p_i=0.01, p_hu_vs = 0.5, p_air = 0.000001): 
    # surfaces
    surface_component = 1 - (1 - p_hu_vs)*N_s*N_p_s*p_i
    
    # ppl breathing
    radius_z_score = (1 / projectiness * r**2 - 18) / 6
    N_v_below_T_c = scipy.stats.norm.cdf(radius_z_score)
    ppl_breathing_component = 1 - (N_p_b * p_i) + p_i * N_p_b * N_v_below_T_c**N_b
    #print("ppl breathing")
    #print(ppl_breathing_component)

    # inhalation-based airborne transmission
    airborne_component = p_air * N_p_b / r**2
    #print("airborne_component")
    #print(airborne_component)
    
    p_youre_infected = 1 - surface_component*ppl_breathing_component + airborne_component
    #print("p_youre_infected")
    #print(p_youre_infected)
    return p_youre_infected
    

## What if the sidewalk is only 4 feet wide -- should I #stayHomeSaveLives?

What if I pass N_p_b = (10, 30, 100) people and they (all cough on me, never cough on me but aren't wearing masks, all wear t-shirt masks, all wear surgical masks)? Corresponding to projectiness = (1, 0.5, 0.3, 0.1).

In [127]:
# situation-specific parameters
N_s = 0 # walk-only: no touching
N_p_s = 0 # walk-only: no touching

r = 4
N_b = 3 # assuming it takes you 4 breaths to pass someone
N_p_b = [10, 30, 100]
projectiness = [1, 0.5, 0.3, 0.1] 

results_grid_r4 = [[SYGOestimator(N_s, N_p_s, p_b, N_b, r, proj) for p_b in N_p_b] for proj in projectiness]

results_grid_r6 = [[SYGOestimator(N_s, N_p_s, p_b, N_b, 6, proj) for p_b in N_p_b] for proj in projectiness]
    
print(results_grid_r4)
print("\n")
print(results_grid_r6)


[[0.09495823453298283, 0.28487470359894845, 0.9495823453298281], [0.0029164159473066514, 0.008749247841920287, 0.029164159473067178], [6.25583201043014e-07, 1.8767496033510865e-06, 6.255832010874229e-06], [6.249999999999999e-07, 1.8749999999999998e-06, 6.2499999999999995e-06]]


[[0.0004047007658397587, 0.0012141022975194983, 0.0040470076583982535], [2.7777777777777776e-07, 8.333333333333332e-07, 2.7777777777777775e-06], [2.7777777777777776e-07, 8.333333333333332e-07, 2.7777777777777775e-06], [2.7777777777777776e-07, 8.333333333333332e-07, 2.7777777777777775e-06]]


## How does "riskiness of the hangout" scale with "length of the hangout"?

Let's try hangouts with different numbers of people: N_p_b = (1, 5, 20). And let N_b = (240, 960, 2880), corresponding to a 15min, 1hr, and 3hr hangout. Try this for projectiness = (1, 0.5, 0.3), corresponding to doomsday (every breath is a cough), just-talking, and talking-with-tshirt-masks. Assume you all stay 6-feet social-distanced.

In [128]:
# situation-specific parameters
N_s = 0 # hangout-only: no touching
N_p_s = 0 # hangout-only: no touching

r = 6
N_b = [8, 240, 960, 2880] # 15min, 1 hour, 3 hours
N_p_b = [1, 5, 30, 100]

results_grid_p1 = [[SYGOestimator(N_s, N_p_s, p_b, b, r, 1, p_i=p_i) for p_b in N_p_b] for b in N_b]

results_grid_p05 = [[SYGOestimator(N_s, N_p_s, p_b, b, r, 0.5, p_i=p_i) for p_b in N_p_b] for b in N_b]

results_grid_p03 = [[SYGOestimator(N_s, N_p_s, p_b, b, 3, 0.5, p_i=p_i) for p_b in N_p_b] for b in N_b]
    
print(results_grid_p1)
print("\n")
print(results_grid_p05)
print("\n")
print(results_grid_p03)


[[2.152437673611489e-05, 0.00010762188368046342, 0.0006457313020827805, 0.002152437673608824], [0.0005538084531458208, 0.002769042265729215, 0.01661425359437562, 0.05538084531458518], [0.001453204927329954, 0.007266024636649659, 0.04359614781989828, 0.14532049273299383], [0.0019591506877099426, 0.009795753438549935, 0.0587745206312995, 0.19591506877099804]]


[[2.7777777777777777e-08, 1.3888888888888888e-07, 8.333333333333332e-07, 2.7777777777777775e-06], [2.7777777777777777e-08, 1.3888888888888888e-07, 8.333333333333332e-07, 2.7777777777777775e-06], [2.7777777777777777e-08, 1.3888888888888888e-07, 8.333333333333332e-07, 2.7777777777777775e-06], [2.7777777777777777e-08, 1.3888888888888888e-07, 8.333333333333332e-07, 2.7777777777777775e-06]]


[[0.001992298611111076, 0.0099614930555556, 0.05976895833333338, 0.1992298611111111], [0.002000111111111113, 0.010000555555555565, 0.06000333333333339, 0.20001111111111106], [0.002000111111111113, 0.010000555555555565, 0.06000333333333339, 0.20001

## How risky is going to Costco vs. going to the corner store?


In [129]:
# situation-specific parameters
# # surfaces
num_items_bought = 20
N_s = num_items_bought*2.5 + 100 # you have to touch each item at least twice. assume 100 misc touches as well

N_people_foot_traffic = [10, 100, 1000, 3600] # everyone who was in the store since the last deep clean
p_t = 0.001 # proportion of store surfaces touched by everyone in the store--assume constant store size
p_t_arr = [0.001, 0.0008, 0.0005, 0.00025] # p_t but assume store size scales with increasing foot traffic

N_p_s_fixed_vol = [p_t * n for n in N_people_foot_traffic]
N_p_s_variable_vol = [p_t_arr[i] * N_people_foot_traffic[i] for i in range(4)]

# # people breathing
r = 6
N_b = 8 
N_p_b = [1, 5, 30, 500]

# fixed-volume store, socially distant
results_grid = [[SYGOestimator(N_s, p_s, p_b, N_b, r, 0.5, p_i) for p_b in N_p_b] for p_s in N_p_s_fixed_vol]
print(results_grid)
print("\n")

# fixed-volume store; not socially distant
results_grid_2 = [[SYGOestimator(N_s, p_s, p_b, N_b, 4, 0.5, p_i) for p_b in N_p_b] for p_s in N_p_s_fixed_vol]
print(results_grid_2)
print("\n")

# volume of store scales with number of people; 
results_grid_3 = [[SYGOestimator(N_s, N_p_s_variable_vol[p_s], p_b, N_b, r, 0.5, p_i) for p_b in N_p_b] for p_s in range(len(N_p_s_variable_vol))]
print(results_grid_3)
print("\n")



[[0.0015000277777777237, 0.0015001388888888348, 0.0015008333333332792, 0.0015138888888888346], [0.015000027777777791, 0.015000138888888901, 0.015000833333333347, 0.015013888888888903], [0.1500000277777778, 0.1500001388888889, 0.15000083333333336, 0.15001388888888892], [0.5400000277777778, 0.5400001388888889, 0.5400008333333334, 0.5400138888888889]]


[[0.0016515896676617039, 0.0022579483383086247, 0.00604769002985202, 0.07729483383086684], [0.015149540977863632, 0.015747704889318103, 0.019486229335908226, 0.08977048893180158], [0.15012905407988236, 0.15064527039941156, 0.15387162239646898, 0.2145270399411486], [0.5400698697079364, 0.5403493485396816, 0.5420960912380891, 0.574934853968151]]


[[0.0015000277777777237, 0.0015001388888888348, 0.0015008333333332792, 0.0015138888888888346], [0.012000027777777788, 0.012000138888888899, 0.012000833333333344, 0.0120138888888889], [0.07500002777777773, 0.07500013888888885, 0.0750008333333333, 0.07501388888888884], [0.13500002777777778, 0.1350001